## Imports

In [1]:
# Imports

import pandas as pd
import numpy as np
import acquire
import prepare
import explore
from statistics import mean



import re
import unicodedata
import pandas as pd
import nltk

import prepare
import acquire

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns



## Call In DF

In [2]:
df = acquire.get_github_geology_results(cached=True)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1841 entries, 0 to 1840
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   language  1841 non-null   object        
 1   content   1841 non-null   object        
 2   date      1841 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 57.5+ KB
None


,language,content,date
0,Jupyter Notebook,hylite\nhylite is an open-source python packag...,2020-11-27 17:31:07.762
1,Python,\n\n\n\n\n\nWebviz subsurface\n\n✨👓 Live demo ...,2020-11-27 17:31:08.305
2,R,"Reforester\nReforester, an R program that:\n\n...",2020-11-27 17:31:08.926
3,JavaScript,Swissgeol NGM\nA Geology 3D viewer\nSwissgeol ...,2020-11-27 17:31:09.532
4,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,2020-11-27 17:31:10.206


In [3]:
df.language.value_counts()

Python              463
Jupyter Notebook    460
R                   232
TypeScript          229
JavaScript          229
Batchfile           228
Name: language, dtype: int64

In [4]:
## Extra Word List
#["accuracy","accurate","acknowledgement","acquired",
#     "adhere","adheres","along","also","accessing","provided",
#     "directory", "datasets","including","pip","problem","pull",
#     "python"]

In [5]:
#Clean
df = prepare.prep_data(df, 'content', extra_words=[], exclude_words=[])
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1841 entries, 0 to 1840
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   language    1841 non-null   object        
 1   content     1841 non-null   object        
 2   date        1841 non-null   datetime64[ns]
 3   clean       1841 non-null   object        
 4   stemmed     1841 non-null   object        
 5   lemmatized  1841 non-null   object        
 6   words       1841 non-null   object        
 7   doc_length  1841 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 129.4+ KB
None


,language,content,date,clean,stemmed,lemmatized,words,doc_length
0,Jupyter Notebook,hylite\nhylite is an open-source python packag...,2020-11-27 17:31:07.762,hylite hylite opensource python package prepro...,hylit hylit opensourc python packag preprocess...,hylite hylite opensource python package prepro...,"[hylite, hylite, opensource, python, package, ...",293
1,Python,\n\n\n\n\n\nWebviz subsurface\n\n✨👓 Live demo ...,2020-11-27 17:31:08.305,webviz subsurface live demo application introd...,webviz subsurfac live demo applic introduct re...,webviz subsurface live demo application introd...,"[webviz, subsurface, live, demo, application, ...",261
2,R,"Reforester\nReforester, an R program that:\n\n...",2020-11-27 17:31:08.926,reforester reforester r program uses logistic ...,reforest reforest r program use logist birthde...,reforester reforester r program us logistic bi...,"[reforester, reforesterprogram, us, logistic, ...",762
3,JavaScript,Swissgeol NGM\nA Geology 3D viewer\nSwissgeol ...,2020-11-27 17:31:09.532,swissgeol ngm geology viewer swissgeol new geo...,swissgeol ngm geolog viewer swissgeol new geol...,swissgeol ngm geology viewer swissgeol new geo...,"[swissgeol, ngm, geology, viewer, swissgeol, n...",120
4,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...,2020-11-27 17:31:10.206,geomodeldweb generates web versions geological...,geomodeldweb gener web version geolog model pr...,geomodeldweb generates web version geological ...,"[geomodeldweb, generates, web, version, geolog...",169


In [6]:
df.language.value_counts()

Python              463
Jupyter Notebook    460
R                   232
TypeScript          229
JavaScript          229
Batchfile           228
Name: language, dtype: int64

## Ave Length of Readme

In [7]:
python_doc_length = round(mean(df[df.language == 'Python'].doc_length), 2)
jupyter_doc_length = round(mean(df[df.language == 'Jupyter Notebook'].doc_length),2)
r_doc_length = round(mean(df[df.language == 'R'].doc_length),2)
javascript_doc_length = round(mean(df[df.language == 'JavaScript'].doc_length),2)
typescript_doc_length = round(mean(df[df.language == 'TypeScript'].doc_length),2)
batchfile_doc_length = round(mean(df[df.language == 'Batchfile'].doc_length),2)

print(f'The average length of the python repositories is {python_doc_length} words')
print(f'The average length of the jupyter repositories is {jupyter_doc_length} words')
print(f'The average length of the r repositories is {r_doc_length} words')
print(f'The average word length of the javascript repositories is {javascript_doc_length} words')
print(f'The average word length of the typescript repositories is {typescript_doc_length} words')
print(f'The average word length of the batchfile repositories is {batchfile_doc_length} words')

The average word length of the python repositories is 215.1
The average word length of the jupyter repositories is 170.57
The average word length of the r repositories is 762
The average word length of the javascript repositories is 120
The average word length of the typescript repositories is 290
The average word length of the batchfile repositories is 303


## Explore List of Words

In [8]:
# Create list of words
python_words = ' '.join(df[df.language=='Python'].lemmatized)
jupyter_words = ' '.join(df[df.language=='Jupyter Notebook'].lemmatized)
r_words = ' '.join(df[df.language=='R'].lemmatized)
typescript_words = ' '.join(df[df.language=='TypeScript'].lemmatized)
javascript_words = ' '.join(df[df.language=='JavaScript'].lemmatized)
batchfile_words = ' '.join(df[df.language=='Batchfile'].lemmatized)
all_words = ' '.join(df.lemmatized)

# remove spaces
python_words = re.sub(r'\s.\s', '', python_words)
jupyter_words = re.sub(r'\s.\s', '', jupyter_words)
r_words = re.sub(r'\s.\s', '', r_words)
typescript_words = re.sub(r'\s.\s', '', typescript_words)
javascript_words = re.sub(r'\s.\s', '', javascript_words)
batchfile_words = re.sub(r'\s.\s', '', batchfile_words)
all_words = re.sub(r'\s.\s', '', all_words)

In [9]:
# wordcounts

python_freq = pd.Series(python_words.split()).value_counts()
jupyter_freq = pd.Series(jupyter_words.split()).value_counts()
r_freq = pd.Series(r_words.split()).value_counts()
typescript_freq = pd.Series(typescript_words.split()).value_counts()
javascript_freq = pd.Series(javascript_words.split()).value_counts()
batchfile_freq = pd.Series(batchfile_words.split()).value_counts()
all_freq = pd.Series(all_words.split()).value_counts()

## What repositories detail work in Geology

In [10]:
# No geology keywords in top 20 most frequent words
python_freq.head(20)

file                2776
test                1392
repository          1391
code                1391
see                 1390
documentation       1389
run                 1159
also                1159
install             1159
gocad               1155
check                928
configuration        928
subsurface           928
webvizsubsurface     928
github               928
webviz               928
registry             928
container            928
pip                  928
application          928
dtype: int64

In [11]:
# Several geology keywords in to 20
jupyter_freq.head(20)

hylite          4176
install         2088
data            1392
python          1392
import          1160
also            1160
gdal            1160
spectral        1160
using           1152
please           928
installation     696
conda            696
minimum          696
mapping          696
pip              696
wavelength       696
directory        696
github           696
variety          696
library          696
dtype: int64

In [12]:
# many geology keywords in top 20
r_freq.head(20)

fossil           4408
simulation       3016
run              3016
sampling         3016
using            2552
diversity        2320
rate             2320
data             2320
tree             2088
time             2088
file             1856
reconstructed    1624
model            1624
true             1624
result           1392
license          1392
record           1392
bias             1160
could            1160
number           1160
dtype: int64

In [13]:
# No geology Content
typescript_freq.head(20)

file             2977
model            2748
start            1832
server           1603
service          1145
see              1145
website           916
graphic           916
conversion        916
npm               916
web               916
provides          916
information       687
readmemd          687
python            687
adding            687
config            687
httplocalhost     687
nvcl              687
borehole          687
dtype: int64

In [14]:
# many geology references
javascript_freq.head(20) 

use          687
viewer       687
see          687
open         687
swisstopo    687
url          458
lava         458
source       458
npm          458
disable      458
limit        458
display      458
cesium       458
define       458
parameter    458
quality      458
swissgeol    458
ngm          458
geology      458
version      458
dtype: int64

In [15]:
# All in spannish, not so helpful to me
batchfile_freq.head(20)

de               7752
la               2280
en               1824
curso            1368
el               1368
del              1140
mapas            1140
web               912
por               912
crear             912
disponibles       912
utilizacion       684
grillas           684
para              684
partir            684
datos             684
contenido         456
argentina         456
multitematica     456
se                456
dtype: int64

In [16]:
all_freq.head(20)

de            7752
file          7609
model         5296
fossil        4408
run           4404
hylite        4176
data          4173
using         4168
see           3918
install       3705
sampling      3016
simulation    3016
also          3012
time          2544
code          2544
python        2542
rate          2320
diversity     2320
script        2304
start         2293
dtype: int64

***
### Focus on Repositories That have Geology Content
 
 - `No`: Python, Typescrpt, Batchfile   
 
 - `Yes`: Jupyter, R, Javascript   

***

In [17]:
# Dataframe to compare most frequent words

# word_counts = (pd.concat([all_freq, python_freq, jupyter_freq, r_freq, typescript_freq, javascript_freq, batchfile_freq], axis=1, sort=True)
#                .set_axis(['all', 'python', 'jupyter', 'r', 'typescript', 'javascript', 'batchfile'], axis=1, inplace=False)
#                .fillna(0)
#                .apply(lambda s: s.astype(int))
#               )

# word_counts.head(20)

word_counts = (pd.concat([jupyter_freq, r_freq, javascript_freq], axis=1, sort=True)
               .set_axis(['jupyter', 'r', 'javascript'], axis=1, inplace=False)
               .fillna(0)
               .apply(lambda s: s.astype(int))
              )

word_counts.head(20)

,jupyter,r,javascript
accessing,0,232,0
according,0,232,0
account,0,232,0
accuracy,228,0,0
accurate,228,0,0
acknowledgement,0,464,0
acquired,232,0,0
across,0,232,0
activate,464,0,0
adapt,0,0,229


In [18]:
# These might be words to excude to make modeling easier
word_counts[word_counts.javascript == word_counts.jupyter].sort_values(by='jupyter').head(10)

,jupyter,r,javascript
accessing,0,232,0
provided,0,696,0
properly,0,232,0
prompt,0,232,0
projectaim,0,232,0
program,0,696,0
produced,0,464,0
produce,0,464,0
probability,0,232,0
proxy,0,1160,0


In [19]:
#These might be words to excude to make modeling easier
word_counts[word_counts.r == word_counts.jupyter].sort_values(by='jupyter', ascending=False).head(10)

,jupyter,r,javascript
directory,696,696,0
different,696,696,0
reference,232,232,0
analysis,232,232,0
need,232,232,0
including,232,232,0
datasets,232,232,0
nolimitfalse,0,0,229
ownterrainfalse,0,0,229
optimization,0,0,229


In [20]:
#These might be words to excude to make modeling easier
word_counts[word_counts.javascript == word_counts.r].sort_values(by='r').head(10)

,jupyter,r,javascript
accuracy,228,0,0
pip,696,0,0
point,232,0,0
pointcloud,232,0,0
polymorphism,232,0,0
preprocessing,696,0,0
problem,228,0,0
processing,460,0,0
pull,232,0,0
python,1392,0,0


In [21]:
***

`Removed Word List`
1. Went back and edited clean to remove numbers

["accuracy","accurate","acknowledgement","acquired",
 "adhere","adheres","along","also","accessing","provided",
 "directory", "datasets","including","pip","problem","pull",
 "python"
]

SyntaxError: invalid syntax (<ipython-input-21-ae35d69d9408>, line 1)

In [ ]:
# visualize the percentage of the term in jupyter, r, javascript
# (word_counts.assign(p_python = word_counts.python/word_counts['all'], 
#                    p_jupyter = word_counts.jupyter/word_counts['all'],
#                    p_r = word_counts.r/word_counts['all'],
#                    p_typescript = word_counts.typescript/word_counts['all'],
#                    p_javascript = word_counts.javascript/word_counts['all'],
#                    p_batchfile = word_counts.batchfile/word_counts['all'])
#  .sort_values(by='all')[['p_python', 'p_jupyter', 'p_r','p_typescript','p_javascript','p_batchfile']]
#  .tail(20)
#  .sort_values('p_jupyter')
#  .plot.barh(stacked=True, figsize = (20,10))
# )

# plt.title("Proportion of 20 most common keywords in GitHub Geology README Files", fontsize = 20)
# plt.show()


In [ ]:
# combine list of word counts into df for further exploration
word_counts = explore.create_df_word_counts(jupyter_freq, r_freq, javascript_freq)
word_counts.head()

***
`Next, compute proportion of each string that for each language.
Because the languages are relatively balanced, we should see about the same proportion as the distibution in the dataset. If we see significantly higher or lower proportion, then that word might be a good feature in predicting that language.
A chi-square test could confirm or deny significance if needed.`
***

In [ ]:
word_counts = explore.word_counts_proportion(word_counts)
word_counts.head(10)

In [ ]:
# visualize the % of the term in each language
explore.proportion_visualization(word_counts)

In [ ]:
# Wordcloud python
plt.figure(figsize=(10,10))
text = jupyter_words
wc = WordCloud(background_color="white",
               max_words=2000, max_font_size=224,
               random_state=42)
wc.generate(text)
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Wordcloud js
plt.figure(figsize=(10,10))
text = javascript_words
wc = WordCloud(background_color="white",
               max_words=2000, max_font_size=224,
               random_state=42)
wc.generate(text)
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Wordcloud R
plt.figure(figsize=(10,10))
text = r_words
wc = WordCloud(background_color="white",
               max_words=2000, max_font_size=224,
               random_state=42)
wc.generate(text)
plt.imshow(wc, interpolation="bilinear")
plt.axis('off')
plt.show()

## Bigrams and Trigrams

In [ ]:
#Create list of bigrams
ham_bigrams = pd.Series(list(nltk.ngrams(ham_words.split(), 2))).value_counts().hea

In [ ]:
# addd code